In [1]:
# import Libraries

import s3fs
import xarray as xr
import pandas as pd

In [2]:
# Connect to AWS S3 storage
fs = s3fs.S3FileSystem(anon=True)

## By downloading the master CSV file enumerating all available data stores, we can interact with the spreadsheet
## through a pandas DataFrame to search and explore for relevant data using the CMIP6 controlled vocabulary:
    
df = pd.read_csv("https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.csv")


In [3]:
#Institutes=["'NCAR'","'NCAR'","'NCAR'","'MOHC'","'E3SM-Project'","'MPI-M'","'MIROC'"] #"''"
#ModelNames=["'CESM1'","'CESM2'","'CESM2-WACCM'","'UKESM1-0-LL'","'E3SM-1-0'","'MPI-ESM1-2-LR'","'MIROC6'"]

Institutes=["'NCAR'"] #"''"
ModelNames=["'CESM2'"]

In [4]:
for iname,mname in zip(Institutes,ModelNames):
    for rname in range(1,2):
        print(mname)
        rstring='r'+str(rname)+'i1p1f1'
        reString="'%s'" % rstring
        #print(reString)
        #print(rname)
        qString="activity_id=='CMIP' & institution_id =="+ iname+" & source_id=="+mname+" & table_id=='Amon' & experiment_id=='historical' &  member_id=="+reString+" & (variable_id=='rsdt' or variable_id=='rsutcs' or variable_id=='rsut' or variable_id=='clt' or variable_id=='clivi' or variable_id=='clwvi' or variable_id=='tas' or variable_id=='psl' or variable_id=='pr' ) "
        #qString="activity_id=='CMIP' & institution_id =="+ iname+" & source_id=="+mname+"  & experiment_id=='historical' &  member_id=="+reString+" & (variable_id=='cod' or variable_id=='cdnc') "       
        #print(iname)
        df_subset = df.query(qString)
        #print(qString)
        for var in df_subset.zstore.values:
            zstore = var
            mapper = fs.get_mapper(zstore)
            # open using xarray
            ds = xr.open_zarr(mapper, consolidated=True)
            #print(ds.variables)
            #for x in ds.data_vars:
            for x in ds.variables:
                if "cell_methods" in ds[x].attrs :
                    #print(x)
                    fname=mname.strip("' '")+"_historical_"+reString.strip("' '")+"_"+x+".nc"
                    print(fname)
                    ds.to_netcdf(path=fname,format='NETCDF4',mode='w')#("CMIP6nc\"+x+".nc")
print('ok')


'CESM2'
CESM2_historical_r1i1p1f1_clivi.nc


PermissionError: [Errno 13] Permission denied: b'/home/dhingmire/CMIP6nc/CESM2_historical_r1i1p1f1_clivi.nc'